<a href="https://colab.research.google.com/github/roberten2608/alura_imersao/blob/main/agente_criacao_atividades_educativas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["secretName"] = userdata.get('secretName')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função agente_buscador
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador_atividades",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especializado em educação infantil. Use a ferramenta de busca do google (google_search)
        para encontrar ideias de atividades para crianças sobre o tópico abaixo.
        Foque em atividades práticas e criativas. Considere apenas atividades recentes e inovadoras.
        """,
        description="Agente que busca ideias educativas no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    atividades = call_agent(buscador, entrada_do_agente_buscador)
    return atividades

In [ ]:
# Função auxiliar para exibir texto em Markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Agente 1: Buscador de Atividades
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador_atividades",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especializado em educação infantil. Usando a ferramenta de busca do Google,
        busque ideias de atividades para crianças sobre o tópico dado.
        Priorize atividades práticas e criativas que geralmente incluem diagramas visuais ou imagens.
        Procure em blogs de professores e sites educacionais que frequentemente compartilham recursos visuais.
        """,
        description="Agente que busca ideias educativas no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    atividades = call_agent(buscador, entrada_do_agente_buscador)
    return atividades

In [ ]:
# Agente 2: Planejador de Atividades
def agente_planejador(topico, atividades_buscadas):
    planejador = Agent(
        name="agente_planejador_atividades",
        model="gemini-2.0-flash",
        instruction="""
        Organize as ideias de atividades encontradas em categorias específicas: atividades manuais, jogos, música e movimento.
        Indique quais materiais são necessários para cada atividade e como aplicar essas atividades em uma aula.
        """,
        description="Agente que planeja atividades educativas"
    )

    entrada_do_agente_planejador = f"Tópico: {topico}\nAtividades encontradas: {atividades_buscadas}"
    plano_de_atividades = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_atividades


In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Atividades Educativas ��")

# Obter tópico do usuário
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer ideias de atividades: ")

if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Vamos criar ideias de atividades para {topico}")

    atividades_buscadas = agente_buscador(topico, data_de_hoje)
    print("\n--- �� Resultado do Agente 1 (Buscador de Atividades) ---\n")
    display(to_markdown(atividades_buscadas))

    plano_de_atividades = agente_planejador(topico, atividades_buscadas)
    print("\n--- �� Resultado do Agente 2 (Planejador de Atividades) ---\n")
    display(to_markdown(plano_de_atividades))

🚀 Iniciando o Sistema de Criação de Atividades Educativas ��
❓ Por favor, digite o TÓPICO sobre o qual você quer ideias de atividades: Atividades numeral 1
Vamos criar ideias de atividades para Atividades numeral 1

--- �� Resultado do Agente 1 (Buscador de Atividades) ---



> Aqui estão algumas ideias de atividades com o numeral 1 que encontrei para crianças:
> 
>  
> Com certeza! Aqui estão algumas ideias de atividades criativas e práticas para ensinar o numeral 1 às crianças, priorizando aquelas com recursos visuais:
> 
>  *  **Colagem do número 1:** Imprima ou desenhe um grande número 1 em uma folha. Peça às crianças para colarem pedacinhos de papel colorido, botões, lantejoulas ou outros materiais sobre o número, criando uma textura interessante.
>  *  **Contagem de objetos:** Reúna diversos objetos pequenos, como brinquedos, frutas ou blocos de montar. Peça às crianças para separarem um único objeto de cada tipo, reforçando o conceito de "um".
>  *  **Desenho e pintura:** Incentive as crianças a desenharem um objeto que represente o número 1, como um sol, uma flor ou uma casa. Elas podem usar tintas, lápis de cor ou canetinhas para colorir seus desenhos.
>  *  **Modelagem com massinha:** Ofereça massinha de modelar e peça às crianças para criarem o número 1. Elas também podem modelar um objeto que represente o número, como uma cobra ou um palito.
>  *  **Jogo da trilha:** Desenhe uma trilha com casas numeradas de 1 a 10. As crianças jogam um dado e avançam o número de casas correspondente. Ao chegarem na casa número 1, elas devem realizar uma tarefa simples, como dar um pulo ou bater palma.
>  *  **Livro sensorial do número 1:** Crie um livro com diferentes texturas e materiais, representando o número 1 em cada página. Por exemplo, uma página pode ter um pedaço de feltro colado em forma de número 1, enquanto outra pode ter um botão grande.
> 
>  **Recursos adicionais:**
> 
>  *  **Vídeos educativos:** Procure por vídeos no YouTube que ensinem o número 1 de forma lúdica e divertida, com músicas e animações.
>  *  **Aplicativos e jogos digitais:** Existem diversos aplicativos e jogos educativos que ajudam as crianças a aprenderem os números de forma interativa.
> 
>  **Dica:** Adapte as atividades à idade e ao nível de desenvolvimento das crianças. O mais importante é tornar o aprendizado divertido e significativo!



--- �� Resultado do Agente 2 (Planejador de Atividades) ---



> ## Planejamento de Atividades com o Numeral 1
> 
> Aqui está um plano de atividades organizadas por categorias, com materiais necessários e sugestões de aplicação em sala de aula:
> 
> **1. Atividades Manuais:**
> 
> *   **Colagem do número 1:**
>     *   **Materiais:**
>         *   Folhas de papel com o número 1 grande impresso ou desenhado.
>         *   Pedaços de papel colorido, botões, lantejoulas, grãos, barbante, etc.
>         *   Cola.
>     *   **Aplicação:**
>         1.  Apresente o número 1 visualmente.
>         2.  Distribua as folhas e os materiais de colagem.
>         3.  Demonstre como colar os materiais sobre o número 1.
>         4.  Incentive a criatividade e a exploração de diferentes texturas.
> *   **Modelagem com massinha:**
>     *   **Materiais:**
>         *   Massinha de modelar de diversas cores.
>         *   Superfícies lisas (mesas ou placas).
>     *   **Aplicação:**
>         1.  Apresente o número 1.
>         2.  Demonstre como modelar o número 1 com a massinha.
>         3.  Incentive as crianças a criarem o número 1 e um objeto que o represente (ex: 1 cobra, 1 flor).
> *   **Livro sensorial do número 1:**
>     *   **Materiais:**
>         *   Folhas de feltro, tecido, papel cartão.
>         *   Cola, tesoura.
>         *   Diferentes materiais com texturas (botões, lixa, algodão, lã, etc.).
>     *   **Aplicação:**
>         1.  Confeccionar um livro, onde cada página representa o número 1 com uma textura diferente.
>         2.  Explore o livro sensorial em sala de aula, permitindo que as crianças toquem e sintam as diferentes texturas enquanto reforçam o conceito do número 1.
> 
> **2. Jogos:**
> 
> *   **Contagem de objetos:**
>     *   **Materiais:**
>         *   Diversos objetos pequenos (brinquedos, frutas, blocos, lápis, etc.).
>         *   Cestas ou recipientes.
>     *   **Aplicação:**
>         1.  Apresente os objetos.
>         2.  Peça para as crianças separarem "um" objeto de cada tipo.
>         3.  Incentive a contagem em voz alta: "Um brinquedo, uma fruta, um bloco..."
> *   **Jogo da trilha:**
>     *   **Materiais:**
>         *   Trilha desenhada com casas numeradas de 1 a 10.
>         *   Dado.
>         *   Marcadores (peças pequenas, botões, etc.).
>     *   **Aplicação:**
>         1.  Explique as regras do jogo.
>         2.  As crianças jogam o dado e avançam na trilha.
>         3.  Ao chegar na casa número 1, realizam uma tarefa (ex: dar um pulo, bater palma).
> 
> **3. Música e Movimento:**
> 
> *   **Música do número 1:**
>     *   **Materiais:**
>         *   Acesso à internet ou CD player.
>         *   Músicas infantis sobre o número 1 (procure no YouTube ou plataformas de streaming).
>     *   **Aplicação:**
>         1.  Apresente a música.
>         2.  Cante e dance com as crianças, enfatizando o número 1 na letra da música.
> *   **Brincadeira "Um Dedinho":**
>     *   **Materiais:** Nenhum.
>     *   **Aplicação:**
>         1.  Cante uma música que envolva levantar "um dedinho". Exemplo: "Um dedinho, um dedinho, vou levantar, vou levantar..."
>         2.  Varie a brincadeira pedindo para levantar um pé, um braço, etc.
> 
> **Recursos Adicionais:**
> 
> *   **Vídeos Educativos:** Utilize vídeos do YouTube que ensinem o número 1 de forma lúdica.
> *   **Aplicativos e Jogos Digitais:** Explore aplicativos e jogos educativos interativos sobre o número 1.
> 
> **Dicas Gerais:**
> 
> *   Adapte as atividades à faixa etária e ao nível de desenvolvimento das crianças.
> *   Use recursos visuais sempre que possível (imagens, cartazes, objetos).
> *   Mantenha as atividades curtas e dinâmicas para manter o interesse das crianças.
> *   Torne o aprendizado divertido e significativo! Incentive a participação e o entusiasmo.
> *   Repita as atividades em diferentes momentos para reforçar o aprendizado.
> 
